In [ ]:
import os
import json
import torchaudio
from datasets import Dataset, Audio
from transformers import WhisperProcessor
import time

In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small")

class CustomASRDataset:
    def __init__(self, data_dir, jsonl_file, processor):
        self.data_dir = data_dir
        self.jsonl_file = jsonl_file
        self.processor = processor
    
    def _load_data(self):
        data = {
            "key": [],
            "audio": [],
            "transcript": []
        }
        with open(self.jsonl_file, "r") as f:
            for line in f:
                if line.strip() == "":
                    continue
                instance = json.loads(line.strip())
                print(instance)
                audio_path = os.path.join(self.data_dir, instance["audio"])
                data["key"].append(instance["key"])
                data["audio"].append(audio_path)
                data["transcript"].append(instance["transcript"])
        return data

    def get_data(self):
        instances = self._load_data()
        print("loaded")
        return instances

data_dir = "/home/jupyter/advanced/audio/"
jsonl_file = "/home/jupyter/advanced/asr.jsonl"
custom_dataset = CustomASRDataset(data_dir, jsonl_file, processor)

# Get data
data = custom_dataset.get_data()


In [ ]:
from transformers import WhisperTokenizer, WhisperFeatureExtractor

In [ ]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
class CustomASRData:
    def __init__(self, data):
        self.data = data

    def _process_audio(self, audio_path):
        waveform, sample_rate = torchaudio.load(audio_path)
        return {
            "array": waveform.squeeze().numpy(),
            "sampling_rate": sample_rate
        }

    def preprocess_audio(self):
        audio_data = []
        i = 0
        for audio_path in self.data["audio"]:
            print("running:", i)
            audio_info = self._process_audio(audio_path)
            audio_data.append(audio_info)
            
            i += 1
        return audio_data
    
    def prepare_dataset(self):
        audio_data = self.preprocess_audio()
        
        self.data["audio"] = audio_data
    
        self.data["input_features"] = []
        for audio_info in audio_data:
            self.data["input_features"].append(feature_extractor(audio_info["array"], sampling_rate=audio_info["sampling_rate"]).input_features[0])

        self.data["labels"] = []
        for transcript in self.data["transcript"]:
            self.data["labels"].append(tokenizer(transcript).input_ids)

        return self.data

custom_asr_dataset = CustomASRData(data)
preped_data = custom_asr_dataset.prepare_dataset()

In [ ]:
new_prep_data = []

# Iterate through the length of 'key'
for i in range(len(preped_data['key'])):
    # Create a dictionary for each index 'i'
    entry = {
        'key': preped_data['key'][i],
        'audio': preped_data['audio'][i],
        'transcript': preped_data['transcript'][i],
        'input_features': preped_data['input_features'][i],
        'labels': preped_data['labels'][i]
    }
    # Append the entry to new_prep_data
    new_prep_data.append(entry)


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
train_data, test_data = train_test_split(new_prep_data, test_size=0.2, random_state=42)

# Create train and test datasets
train_dataset = [{"input_features": entry["input_features"], "labels": entry["labels"]} for entry in train_data]
test_dataset = [{"input_features": entry["input_features"], "labels": entry["labels"]} for entry in test_data]


In [ ]:
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

model.generation_config.language = "english"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)


In [ ]:
print(model)


In [ ]:
for param in model.parameters():
    param.requires_grad = False
for param in model.proj_out.parameters():
    param.requires_grad = True

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./wsp-trained",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

# Train model
trainer.train()


In [ ]:
kwargs = {
    "language": "en",
    "model_name": "whisp-small-hw",
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}


In [ ]:
trainer.push_to_hub(**kwargs)


In [5]:
from huggingface_hub import notebook_login, logout

In [6]:
notebook_login()

In [ ]:
logout

In [1]:
!pip install huggingface_hub


In [6]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperTokenizer, WhisperFeatureExtractor

In [10]:
model_name = "Hwww23/wsp-trained"
model = WhisperForConditionalGeneration.from_pretrained(model_name, use_auth_token=True)


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:3027: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [11]:
print(model)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        

In [13]:
model.save_pretrained("/home/jupyter/til-team-33/asr_test/model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


In [1]:
import os
import json
import jiwer
import torch
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline, WhisperFeatureExtractor, WhisperTokenizer
from tqdm import tqdm
import base64
import io

In [2]:
# Define a function to convert numbers to words
def number_to_words(number):
    number_mapping = {
        '0': 'zero', '1': 'one', '2': 'two', '3': 'three', '4': 'four',
        '5': 'five', '6': 'six', '7': 'seven', '8': 'eight', '9': 'nine'
    }
    return ' '.join([number_mapping[char] for char in number])

# Modify the generated transcript
def modify_transcript(transcript):
    modified_transcript = ""
    i = 0
    while i < len(transcript):
        if transcript[i].isdigit():
            number = ""
            while i < len(transcript) and transcript[i].isdigit():
                number += transcript[i]
                i += 1
            modified_transcript += number_to_words(number)
        else:
            modified_transcript += transcript[i]
            i += 1
    return modified_transcript


In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

processor = WhisperProcessor.from_pretrained("openai/whisper-small")
# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
# model_path = "/home/jupyter/til-team-33/asr_test/model"
model_path = "/home/jupyter/til-team-33/asr_test/wsp-trained"
model = WhisperForConditionalGeneration.from_pretrained(model_path)
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="en", task="transcribe")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    chunk_length_s=30,
    device=device,
    tokenizer=tokenizer,
    feature_extractor=feature_extractor,
)

audio_dir = '/home/jupyter/advanced/audio'
transcript_file = '/home/jupyter/advanced/asr.jsonl'

transcripts = {}
with open(transcript_file, 'r') as f:
    for line in f:
        data = json.loads(line.strip())
        transcripts[data["audio"]] = data["transcript"]
        
# Get a list of audio files
audio_files = [f for f in os.listdir(audio_dir) if f.endswith('.wav')]
    
batch_size = 16  
for i in tqdm(range(0, len(audio_files), batch_size)):
    batch_files = audio_files[i:i + batch_size]
    batch_audio_data = []
    ground_truths = []

    for filename in batch_files:
        audio_path = os.path.join(audio_dir, filename)
        with open(audio_path, 'rb') as audio_file:
            audio_bytes = audio_file.read()
            audio_bytes = base64.b64encode(audio_bytes).decode("ascii")
            audio_data = base64.b64decode(audio_bytes)
            audio_file = io.BytesIO(audio_data)
            waveform, sample_rate = torchaudio.load(audio_file)
            audio_data = {"array": waveform.squeeze().numpy(), "sampling_rate": sample_rate}
            batch_audio_data.append(audio_data["array"])
            ground_truths.append(transcripts.get(filename))

    results = pipe(batch_audio_data, batch_size=batch_size)

    for result, ground_truth in zip(results, ground_truths):
        generated_transcript = result["text"]
        modified_transcript = modify_transcript(generated_transcript)

        print("Modified transcript:", modified_transcript)
        print("Ground truth transcript:", ground_truth)

        wer = jiwer.wer(ground_truth, modified_transcript)
        cer = jiwer.cer(ground_truth, modified_transcript)

        print("WER:", wer)
        print("CER:", cer)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0%|          | 1/219 [00:03<12:43,  3.50s/it]

Modified transcript:  Turrets, prepare to deploy electromagnetic pulse. Heading zero six five, target is grey and white fighter jet. Engage when ready.
Ground truth transcript: Turret, prepare to deploy electromagnetic pulse. Heading zero six five, target is grey and white fighter jet. Engage when ready.
WER: 0.05
CER: 0.0078125
Modified transcript:  engage yellow drone with surface-to-air missiles heading two three five.
Ground truth transcript: Engage yellow drone with surface-to-air missiles, heading two three five.
WER: 0.2
CER: 0.0273972602739726
Modified transcript:  Turret Bravo, engage the target, white, green, and silver cargo aircraft, heading zero five five with surface-to-air missiles. Prepare for launch.
Ground truth transcript: Turret Bravo, engage the target, white, green, and silver cargo aircraft, heading zero five five with surface-to-air missiles. Prepare for launch.
WER: 0.0
CER: 0.0
Modified transcript:  Control tower to turrets, deploy EMP towards green, black, an

  1%|          | 2/219 [00:06<12:25,  3.43s/it]

Modified transcript:  Control tower to A.I. defense turrets, deploy surface-to-air missiles, heading one six five, target silver light aircraft. Stand by for further instructions.
Ground truth transcript: Control Tower to AI Defense Turrets, deploy surface-to-air missiles, heading one six five, target silver light aircraft. Standby for further instructions.
WER: 0.2857142857142857
CER: 0.03896103896103896
Modified transcript:  Air defense turret, target the black cargo aircraft at heading two zero five. Deploy surface-to-air missiles. Engage and destroy the target.
Ground truth transcript: Air defense turret, target the black cargo aircraft at heading two zero five. Deploy surface-to-air missiles. Engage and destroy the target.
WER: 0.0
CER: 0.0
Modified transcript:  Control to turrets, prepare to deploy the machine gun to target the purple missile at heading three zero zero. Repeat, deploy machine gun to heading three zero zero. Engage the purple missile. Over.
Ground truth transcript

  1%|▏         | 3/219 [00:10<12:37,  3.51s/it]

...

Modified transcript:  Turret Bravo, engage target, yellow, purple, and black fighter jet at heading two two zero with machine gun. Support turrets, standby for further orders.
Ground truth transcript: Turret Bravo, engage target, yellow, purple, and black fighter jet at heading two two zero with machine gun. Support turrets, stand by for further orders.
WER: 0.08
CER: 0.006493506493506494
Modified transcript:  Control to turrets, deploy EMP tool on heading three five five, targeting black commercial aircraft. Engage and neutralize the threat. Over.
Ground truth transcript: Control to turrets, deploy EMP tool on heading three five five, targeting black commercial aircraft. Engage and neutralize the threat. Over.
WER: 0.0
CER: 0.0
Modified transcript:  Engage the purple, blue, and grey drone at heading one zero five. Deploy anti-air artillery.
Ground truth transcript: Engage the purple, blue, and grey drone at heading one zero five. Deploy anti-air artillery.
WER: 0.0
CER: 0.0
Modified t

 16%|█▌        | 35/219 [02:39<15:28,  5.05s/it]

Modified transcript:  Control to air defense turrets, we have a brown commercial aircraft at heading zero six zero. Deploy interceptor jets to engage the target. Repeat, deploy interceptor jets to engage the target. Over.
Ground truth transcript: Control to air defense turrets, we have a brown commercial aircraft at heading zero six zero. Deploy interceptor jets to engage the target. Repeat, deploy interceptor jets to engage the target. Over.
WER: 0.0
CER: 0.0
Modified transcript:  Control tower to turrets, deploy electromagnetic pulse heading two one zero, target brown, yellow, and white light aircraft.
Ground truth transcript: Control Tower to Turrets, deploy electromagnetic pulse heading two one zero, target brown, yellow, and white light aircraft.
WER: 0.1111111111111111
CER: 0.016129032258064516
Modified transcript:  Turret Bravo, track and engage the brown helicopter at heading one three five with surface-to-air missiles. Take the shot when ready. Turret Bravo, the target is now 

 16%|█▌        | 35/219 [02:40<14:01,  4.58s/it]


KeyboardInterrupt: 